In [3]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Example data for the first choropleth map
# data1 = dict(
#     # type = 'choropleth_mapbox',
#     locations = ['USA', 'CAN', 'MEX'],
#     z = [1, 2, 3],
#     locationmode = 'country names',
#     colorscale = 'Blues',
#     colorbar_title = 'Data 1'
# )

import numpy as np
import pandas as pd
from shapely.geometry import Polygon
import geopandas as gpd
import pyproj
pyproj.datadir.set_data_dir('/Users/jm/Code/mapgpt-mvp/.conda/share/proj')
import matplotlib.pyplot as plt
import os
import sys
from dotenv import load_dotenv
import pandas as pd
import pydeck as pdk
from IPython.display import display
sys.path.append('../src')

from infrastructure.gpt4 import GPT4
from infrastructure.postgres_db import PostgresDB
from application.maps.choropleth_map import ChoroplethMap
import plotly.graph_objects as go

load_dotenv()

db_name = os.environ.get("DB_NAME")
db_user = os.environ.get("DB_USER")
db_password = os.environ.get("DB_PASSWORD")


db = PostgresDB(db_name=db_name, db_user=db_user, db_password=db_password)
comunas = db.run_gpd_query("SELECT * FROM comuna")

# Function to extract coordinates
def get_polygon_coordinates(geom):
    if geom.type == 'Polygon':
        return [(x, y) for x, y in geom.exterior.coords]
    elif geom.type == 'MultiPolygon':
        return [[(x, y) for x, y in polygon.exterior.coords] for polygon in geom.geoms][0]
    else:
        return None

comunas = comunas.to_crs(epsg=4326)
np.random.seed(9)
comunas = comunas.assign(
    lat = comunas.geom.centroid.y,
    lon = comunas.geom.centroid.x,
    coordinates = comunas.geom.apply(lambda geom: [list(coord) for coord in geom.geoms[0].exterior.coords]),
    coordinates2 = comunas.geom.apply(get_polygon_coordinates),
    score=lambda _df: np.random.randint(0, 101, size=len(_df)),
    score1=lambda _df: np.random.randint(0, 101, size=len(_df)),
    score2=lambda _df: np.random.randint(0, 101, size=len(_df)),
    score3=lambda _df: np.random.randint(0, 101, size=len(_df)),
    # geom = comunas.geom.simplify(10, preserve_topology=True)


    
)
# comunas.geom = comunas.geom.simplify(10, preserve_topology=True)
comunas = comunas.to_crs(epsg=3857)
c = comunas.set_index("codigo").head(10)
c.geom = c.geom.simplify(10, preserve_topology=True)
d = comunas.set_index("codigo").tail(10)
d.geom = d.geom.simplify(10, preserve_topology=True)



/Users/jm/Code/mapgpt-mvp/.conda/lib/python3.11/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()
/Users/jm/Code/mapgpt-mvp/.conda/lib/python3.11/site-packages/geopandas/io/sql.py:185: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
/Users/jm/Code/mapgpt-mvp/.conda/lib/python3.11/site-packages/geopandas/io/sql.py:473: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(spatial_ref_sys_sql, con)
/var/folders/8j/bf4mjlj94czf88p79v8gb24w0000gp/T/ipykernel_24506/2590278563.py:56: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSe

In [4]:
gpt4 = GPT4()

In [5]:
x = gpt4.get_embedding("hola")
x

array([ 0.0286979 , -0.03164996, -0.01430055, ..., -0.01932148,
       -0.01322378, -0.01555881])

In [6]:
x.shape

(1536,)

In [14]:
for i, row in pd.read_sql("SELECT id, nombre FROM candidato", con=db.engine).iterrows():
    update_query = "UPDATE your_table SET your_column = %s WHERE id = %s"
    cur.execute(update_query, (value, row_id))

,id,nombre
0,1,Apruebo
1,2,Rechazo
2,3,Votos Nulos
3,4,Votos En Blanco
4,5,Patricio Ferreira Rivera
...,...,...
142166,152306,Mauricio Andres Contreras Diaz
142167,152307,Francisco Javier Diaz Martinez
142168,152308,Veronica Nogueira Boettiger
142169,152309,Ruben Esteban Mendez Venegas


In [ ]:

# Connect to the PostgreSQL database
with db.conn.cursor() as cur:
    # Create a cursor object to execute SQL queries
    # Define a dictionary with the row values to be updated
    row_values = {
        1: 'value1',
        2: 'value2',
        3: 'value3'
    }

    # Iterate over the dictionary and execute an UPDATE query for each row
    for row_id, value in row_values.items():
        update_query = "UPDATE your_table SET your_column = %s WHERE id = %s"
        cur.execute(update_query, (value, row_id))

    # Commit the changes to the database
    conn.commit()

    # Close the cursor and the connection
    cur.close()
    conn.close()